# Clustering (Unsupervised Learning)

## Overview of Clustering with K-Means

### Overview of Functions

- ```mutate(df, across(everything(), function))```
    - Allows us to apply a transformation to each column.
    - 2nd argument requires the ```across()``` function which makes it easy to apply the same transformation to multiple columns, allowing you to use ```select()``` semantics inside in "data-masking" functions like ```summarise()``` and ```mutate()```.
        - 1st argument requires the ```everything()``` function which selects all variables.
        - Requires the sepcific function applied to each column of the dataframe.
- ```rowwise()```
    - Creates special type of grouping where each group consists of a single row.
- ```kmeans(scaled_df, nstart = ..., centers = ...)```
    - 1st argument requires the scaled dataframe.
    - ```nstart``` specifies the number of restarts to counter the bad starts in clustering.
    - center specifies the number of clusters.
- ```augment(kmeans_object, original_data)```
    - takes in the model and the original dataframe, and returns a data frame with the data and the cluster assignemnts for each point.
    - helps identify different clusters.
- ```glance(kmeans_object)```
    - obtains clustering statistics (including WSSD) for the K-menas clustering object.
    - can be nested int he ```map()``` function to obtain clustering statistics for each kmeans object.
- ```unnest(glanced)```
    - unpacksdataframe into simpler column data frames.
    - used because clustering statistics are put in a dataframe.

### Overview & Error Calculations

The K-means algorithm groups data into K clusters. It starts with an initial clustering of the data, and then iteratively improves it by making adjustments to the assignment of data to clusters until it cannot improve any further.

We measure the quality of the cluster by its ***within-cluster sum-of-squared-distances (WSSD)***. This computation involves 2 steps. *NOTE: WSSD and $S^2$ are used interchangeably.*
1. We find the cluster centers by computing the mean of each variable over data points in the cluster. 
2. Add up the squared distance between each point in the cluster and the cluster center. (This is done using euclidean distance formula: $\text{Distance} = \sqrt((a_x-b_x)^2+(a_y-b_y)^2)$).

<html>
<div style="display:flex; flex-direction:row;">
    <div>
        <h4>Step 1</h4>
        <img src="media/wssd-1-1.png">
        <img src="media/wssd-1-2.png">
    </div>
    <div>
        <h4>Step 2</h4>
        <img src="media/wssd-2-1.png">
        <img src="media/wssd-2-2.png">
    </div>
</div>
</html>

Then, the larger the value $S^2$, the more spread out the cluster is. (Large $S^2$ means that poitns are far from the cluster center). Notably, "large" is relative to ***both the scale of the variables for clustering and the number of points in the cluster***.

After calculating WSSD, we sum them together to get the *total WSSD*.

<img src="media/total_wssd.png" width="300px">

### Clustering Algorithm

We start by picking K, and randomly assigning a roughly equal number of observations to each of the K clusters.

K-means does two steps to minimize the total WSSD:
1. **Center Update**: Compute the center of each cluster.
2. **Label Update**: Reassign each data point to the cluster with the nearest center.

The two steps are repeated until the cluster assignments no longer change.

<img src="media/clustering-algorithm.png" width="400px">

### Random Starts

K-means can get "stuck" in a bad solution.

<img src="media/k_means_stuck.png" width="400px">

To solve this problem with clustering, we should randomly re-initialize the labels a few times, run K-means for each initialization, and pick the clustering data that has the lowest final total WSSD.

### Choosing K

We perform clustering for a variety of possible 

- If K is chosen too small, then multiple clusters would get grouped together (large WSSD)
- If K is chosen too large, then clusters will get subdivided (decreases WSSD by a diminishing amount)

<img src="media/total_wssd_vs_no_clusters.png" width="300px">

## Data Pre-Processing for K-Means

Because K-means clustering uses straight-line distance to decide which points are similar to each other. Therefore, it is very important to scale the variables.

We standardize our data before clustering, which ensures that each variable has a ***mean of 0 and standard deviation of 1***. The ```scale``` function in R can be used to do this.


```r
standardized_data <- not_standardized_data %>%
                     mutate(across(everything(), scale))
```

## Performing K-Means in R

We use the ```kmeans``` function to perform Kmeans clustering:

```r
penguin_clust <- kmeans(standardized_data, centers = 3)
```
<img src="media/kmean_returned.png" width="300px">

The data returned has a lot of information that can be used to 1) visualize cluster, 2) pick K, 3) evaluate the total WSSD.

Using the ```broom``` package we can get the data in a tidy format.

```r
library(broom)
clusted_data <- augment(penguin_clust, standardized_data)
```

> The ```augment``` function takes the standardized data, and adds a 'cluster' column which ```augment``` interpretted from what ```kmeans``` returns.

<img src="media/broom_cleaned_cluster.png" width="300px">

### Visualize Cluster

Now, we can make a visualization of the cluster assignments for each point.

```r
cluster_plot <- cluster_data %>%
                ggplot(aes(x = flipper_length_mm, y = bill_length_mm, color = .cluster), size = 2) +
                    geom_point() +
                    labs(x = "Flipper Length (standardized)", 
                         y = "Bill Length (standardized)", 
                         color = "Cluster") + 
                scale_color_manual(values = c("dodgerblue3", "darkorange3", "goldenrod1")) + 
                theme(text = element_text(size = 12))

cluster_plot
```

<img src="media/cluster_visual.png" width="300px">

### Finding K (NOT Cross-Validation)

We can use ```glance``` to find the total WSSD (```tot.withinss```) from our clustering using ```broom```'s ```glance``` function.

```r
glance(penguin_clust)
```

<img src="media/tot_withinss.png" width="200px">

Then, we create a data frame with column K for all K values we want to test.
```r
penguin_clust_ks <- tibble(k = 1:9)
```

Then, we use ```rowwise``` + ```mutate``` to apply the kmeans function with each row to each K. However, given that the ```kmeans``` function returns a model object to us (not a vector), we will need to store the results in a list column.
```r
penguin_clusts_ks <- tibble(k = 1:9) %>%
                      rowwise() %>%
                      mutate(penguin_clusts = list(kmeans(standardized_data, k)))
```

Looking at the data frame ```penguin_clust_ks```, we see that it has two columns:

<img src="media/penguin_clust_ks.png" width="200px">

<br>

> Using ```pull```, we can get a column in the data frame, then using ```pluck``` we can get the row value, ```kmeans``` data from our dataframe.
> ```r
> penguin_clusts_ks %>%
> pull(penguin_clusts) %>%
> pluck(1)   
> ```

<br>

Next, we use ```mutate``` again to apply ```glance``` to each of the K-means clustering objects to get the clustering statistics (including WSSD). The output is a dataframe, so we need to create another list column. This results in a complex data frame with 3 columns, one for K, one for K-means, and one for the clustering statistics:
```r
penguin_clust_ks <- tibble(k = 1:9) %>%
                    rowwise() %>%
                    mutate(penguin_clusts = list(kmeans(standardized_data, k)),
                           glanced = list(glance(penguin_clusts)))
```

<img src="media/penguin_clusts_ks.png" width="300px">

Notably, each glanced object looks like this:

<img src="media/glanced_object.png" width="300px">

Given that each item in the list column is a data frame, we will need to use ```unnest``` function to unpack the data frames into simpler column data types.

```r
clustering_statistics <- penguin_clust_ks %>%
                         unnest(glanced)
```

<img src="media/clustering_stats.png" width="300px">


Now that we have the ```tot.withinss``` and ```k``` as columns in a data frame, we can make a line plot and search for the "elbow" to find whiich value of K to use.

```r
elbow_plot <- ggplot(clustering_statistics, aes(x = k, y = tot.withinss)) +
                  geom_point() +
                  geom_line() +
                  xlab("K") +
                  ylab("Total within-cluster sum of squares") +
                  scale_x_continuous(breaks = 1:9) + 
                  theme(text = element_text(size = 12))
```

<img src="media/wssd_vs_k.png" width="300px">

# Bad Initialization

Now, why did the WSSD increase for K = 8? This is because we had an unlucky initialization causing bad clustering. We use ```nstart``` to help prevent this. When we do this, K-means clustering will be performed the number of times specified by the ```nstart``` argument.

```r
penguin_clust_ks <- tibble(k = 1:9) |>
                    rowwise() |>
                    mutate(penguin_clusts = list(kmeans(standardized_data, nstart = 10, k)),
                           glanced = list(glance(penguin_clusts)))

clustering_statistics <- penguin_clust_ks |>
                         unnest(glanced)

elbow_plot <- ggplot(clustering_statistics, aes(x = k, y = tot.withinss)) +
                     geom_point() +
                     geom_line() +
                     xlab("K") +
                     ylab("Total within-cluster sum of squares") +
                     scale_x_continuous(breaks = 1:9) + 
                     theme(text = element_text(size = 12))
```

<img src="media/tots_nstart.png" width="300px">